In [3]:
!pip install -U sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.1/799.1 kB 7.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 2.3 MB/s eta 0:00:0000:01
  Using cached importlib_metadata-4.13.0-py3-none-any.whl (23 kB)
  Using cached tblib-1.7.0-py2.py3-none-any.whl (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 44.0 MB/s eta 0:00:0000:0100:01
  Created wheel for sagemaker: filename=sagemaker-2.161.0-py2.py3-none-any.whl size=1077134 sha256=ce0e2f0ef23c57d33e5351812c51a03e09fbdc37463b69a875f28dbded4c286e
  Stored in directory: /root/.cache/pip/wheels/1d/e1/53/227e82641c35cfa39aa08f5ac8990f938640294d274d35b07b
Successfully built sagemaker
  Attempting uninstall: tblib
    Found existing installation: tblib 1.6.0
    Uninstalling tblib-1.6.0:
      Successfully uninstalled tblib-1.6.0
  Attempting uninstall: importlib-metadata
   

In [4]:
import sys

import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()
default_bucket = sagemaker_session.default_bucket()
model_package_group_name = f"AbaloneModelPackageGroupName"

In [5]:
!mkdir -p data

In [6]:
print(default_bucket)

sagemaker-us-east-1-356585319840


In [7]:
local_path = "data/abalone-dataset-batch"

s3 = boto3.resource("s3")
s3.Bucket(f"sagemaker-servicecatalog-seedcode-{region}").download_file(
    "dataset/abalone-dataset-batch", local_path
)

base_uri = f"s3://{default_bucket}/abalone"
batch_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path,
    desired_s3_uri=base_uri,
)
print(batch_data_uri)

s3://sagemaker-us-east-1-356585319840/abalone/abalone-dataset-batch


In [35]:
from sagemaker.transformer import Transformer


transformer = Transformer(
    model_name='Model-zC3V5WAx4uYx',
    instance_type="ml.m5.large",
    instance_count=1,
    output_path=f"s3://{default_bucket}/AbaloneTransform",
)

In [37]:
transformer.transform(data=batch_data_uri)
transformer.wait()

INFO:sagemaker:Creating transform job with name: Model-zC3V5WAx4uYx-2023-06-03-10-56-37-268


....................................
[2023-06-03:11:02:36:INFO] No GPUs detected (normal if no gpus installed)
[2023-06-03:11:02:36:INFO] No GPUs detected (normal if no gpus installed)
[2023-06-03:11:02:36:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2023-06-03 11:02:36 +0000] [15] [INFO] Startin

In [46]:
bucket = sagemaker_session.default_bucket()
s3_list_outputs = sagemaker_session.list_s3_files(bucket, 'AbaloneTransform')

In [50]:
# Read in each of the batch outputs  
# Note - the outputs are in json format.  
import pandas as pd
batch_outputs = []
for m in s3_list_outputs:
    temp_file = pd.read_csv('s3://{}/{}'.format(bucket, m))
    print(temp_file)
    batch_outputs.append(temp_file)
    
batch_outputs = pd.concat(batch_outputs)

#Print the outputs
batch_outputs 

      8.235637664794922
0              7.945878
1             10.220342
2              9.644837
3              8.355701
4              8.388449
...                 ...
4171          14.270259
4172           9.321432
4173          14.957077
4174          10.439097
4175          10.498038

[4176 rows x 1 columns]


,8.235637664794922
0,7.945878
1,10.220342
2,9.644837
3,8.355701
4,8.388449
...,...
4171,14.270259
4172,9.321432
4173,14.957077
4174,10.439097
